# MorphSeq Annotation Pipeline Tutorial

## Prerequisites
- SAM annotation file (`*_sam_annotations.json`)
- Python environment with required packages
- Basic understanding of zebrafish embryo experiments

## Welcome to MorphSeq! 🐠

In [ ]:
import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional

from streamlined_embryo_metadata import EmbryoMetadata
from annotation_batch import AnnotationBatch
from schema_manager import SchemaManager

import pandas as pd
import matplotlib.pyplot as plt

print("🐠 Welcome to MorphSeq! Let's annotate some embryos!")

## 1. Setup and Initialization

In [ ]:
sam_path = Path("data/experiment_001_sam_annotations.json")
metadata_path = Path("data/experiment_01_embryo_metadata.json")

In [ ]:
metadata = EmbryoMetadata(
    sam_annotation_path=sam_path,
    embryo_metadata_path=metadata_path,
    gen_if_no_file=True,
    verbose=True
)

print(f"✅ Loaded metadata with {metadata.embryo_count} embryos and {metadata.snip_count} snips")

## 2. Explore Data Structure

In [ ]:
entity_counts = metadata.get_entity_counts()
print("📊 Entity breakdown:")
for entity_type, count in entity_counts.items():
    print(f"  - {entity_type}: {count}")

In [ ]:
sample_embryo_id = list(metadata.data["embryos"].keys())[0]
print(f"\n🔍 Sample embryo structure ({sample_embryo_id}):")
print(json.dumps(metadata.data["embryos"][sample_embryo_id], indent=2)[:500] + "...")

In [ ]:
experiments = set()
for embryo_id in metadata.data["embryos"]:
    exp_id = embryo_id.split("_")[0]
    experiments.add(exp_id)
print(f"📹 Available experiments: {sorted(experiments)}")

## 🎯 Exercise 1: Entity Explorer
Write a function to extract and count all unique videos from embryo IDs
Hint: Video ID is the second component (e.g., `exp001_v001_e001` → `v001`)

In [ ]:
def count_videos_per_experiment(metadata):
    video_counts = {}
    for embryo_id in metadata.data["embryos"]:
        video_id = embryo_id.split("_")[1]
        video_counts[video_id] = video_counts.get(video_id, 0) + 1
    return video_counts

print("✅ Function ready. Use `count_videos_per_experiment(metadata)` to run.")

## 2.2 Navigate Embryo-Snip Relationships

In [ ]:
embryo_id = "exp001_v001_e001"
snips = metadata.get_available_snips(embryo_id)
print(f"📸 Embryo {embryo_id} has {len(snips)} snips:")
print(f"  First 5: {snips[:5]}")

In [ ]:
if snips:
    sample_snip = snips[0]
    parent_embryo = metadata.get_embryo_id_from_snip(sample_snip)
    print(f"\n🔗 Snip {sample_snip} belongs to embryo {parent_embryo}")

In [ ]:
if len(snips) > 10:
    frame_numbers = []
    for snip in snips[:20]:
        frame_num = int(snip.split("_s")[1])
        frame_numbers.append(frame_num)
    
    plt.figure(figsize=(10, 3))
    plt.plot(frame_numbers, 'o-')
    plt.xlabel("Snip Index")
    plt.ylabel("Frame Number")
    plt.title(f"Temporal Sequence for {embryo_id}")
    plt.show()

## 3. Basic Annotations
Remember: Phenotype → Genotype → Treatment

In [ ]:
phenotypes = metadata.schema_manager.get_phenotypes()
print("🔬 Available phenotypes:")
for name, info in phenotypes.items():
    print(f"  - {name}: {info['description']}")

## 3.1 Adding Phenotypes

In [ ]:
target_embryo = list(metadata.data["embryos"].keys())[0]
metadata.add_phenotype(
    target_embryo,
    phenotype_name="EDEMA",
    frames="0200:0400",
    confidence="high",
    author="tutorial_user"
)
print(f"✅ Added EDEMA to {target_embryo}")

## 3.2 Adding Genotypes

In [ ]:
metadata.add_genotype(
    target_embryo,
    gene_name="tmem67",
    allele="e1127",
    zygosity="homozygous",
    author="tutorial_user"
)
print(f"✅ Added tmem67-e1127 to {target_embryo}")

# Try to add another genotype (will fail - single genotype rule!)
try:
    metadata.add_genotype(
        target_embryo,
        gene_name="lmx1b",
        allele="sa2187"
    )
except ValueError as e:
    print(f"❌ Expected error: {e}")
    print("   → Use overwrite_genotype=True to change existing genotype")

## 3.3 Working with Treatments

In [ ]:
treatments = metadata.schema_manager.get_treatments()
print("💊 Available treatments:")
for name, info in treatments.items():
    print(f"  - {name}: {info['description']} ({info['type']})")

In [ ]:
treated_embryo = list(metadata.data["embryos"].keys())[3]
metadata.add_treatment(
    treated_embryo,
    treatment_name="PTU",
    dosage="200μM",
    timing="24hpf-48hpf",
    author="tutorial_user",
    notes="Standard pigmentation blocking"
)
print(f"✅ Added PTU treatment to {treated_embryo}")

In [ ]:
metadata.add_treatment(
    treated_embryo,
    treatment_name="heat_shock",
    timing="30hpf for 1hr at 37°C",
    author="tutorial_user"
)
print("✅ Added second treatment - compound experiment!")

In [ ]:
control_embryo = list(metadata.data["embryos"].keys())[4]
metadata.add_treatment(
    control_embryo,
    treatment_name="DMSO",
    dosage="0.1%",
    timing="24hpf-48hpf",
    author="tutorial_user",
    notes="Vehicle control for PTU experiment"
)
print(f"✅ Added DMSO control to {control_embryo}")

## 3.4 Quality Control & Flags

In [ ]:
sample_embryo = list(metadata.data["embryos"].keys())[0]
sample_snip = metadata.get_available_snips(sample_embryo)[0]

metadata.add_flag(
    sample_snip,
    flag_type="MOTION_BLUR",
    level="snip",
    priority="medium",
    description="Tail movement during acquisition"
)

metadata.add_flag(
    "exp001_v001",  # Video ID
    flag_type="NONZERO_SEED_FRAME",
    level="video",
    priority="high",
    description="Recording started late"
)
print("✅ Flagged snip and video")

## 4. Batch Annotations
The power move — annotate multiple embryos at once!

In [ ]:
batch = AnnotationBatch(
    author="tutorial_user",
    description="Demo batch for multiple embryos"
)

embryo_list = list(metadata.data["embryos"].keys())[:5]  # First 5

for i, embryo_id in enumerate(embryo_list):
    # Alternate genotypes
    genotype = "tmem67" if i % 2 == 0 else "WT"
    batch.add_genotype(embryo_id, genotype, zygosity="homozygous")
    
    # Add phenotypes
    if genotype == "tmem67":
        batch.add_phenotype(embryo_id, "EDEMA", frames="0200:0400")
        batch.add_phenotype(embryo_id, "CONVERGENCE_EXTENSION", frames="0100:0300")
    
    # Add QC flags
    batch.add_flag(embryo_id, "MOTION_BLUR", level="snip", priority="medium")

print("📦 Batch preview:")
print(batch.preview())

## 7. Quality Control
Keep your data clean and trustworthy!

In [ ]:
coverage = metadata.validate_genotype_coverage()
print(f"📊 Genotype coverage: {coverage['missing_count']} embryos missing genotypes")

## 🎯 Exercise 2: Annotation Detective
Find all embryos with both genotype and phenotype
Bonus: Count how many have treatments too!

In [ ]:
def find_fully_annotated_embryos(metadata):
    results = []
    for embryo_id, data in metadata.data["embryos"].items():
        has_genotype = data.get("genotype") is not None
        has_phenotypes = len(data.get("phenotypes", [])) > 0
        has_treatment = len(data.get("treatments", [])) > 0
        
        if has_genotype and has_phenotypes:
            results.append({
                "embryo_id": embryo_id,
                "genotype": data["genotype"],
                "phenotypes": data["phenotypes"],
                "treatments": data["treatments"],
                "has_treatment": has_treatment
            })
    return results

print("✅ Function ready. Use `find_fully_annotated_embryos(metadata)` to run.")

## 🎯 Exercise 4: Data Analyzer
Create a summary report showing phenotype penetrance by genotype
Hint: Group by genotype, count phenotypes, calculate percentages

In [ ]:
def calculate_penetrance(metadata):
    # TODO: Group by genotype, count phenotypes, return percentages
    # Example return: {"tmem67": {"EDEMA": 0.85, "CONVERGENCE": 0.67}}
    pass

print("✅ Function ready. Implement logic for penetrance calculation.")

## 11. Best Practices
📚 Annotation Guidelines

In [ ]:
print("📚 ANNOTATION BEST PRACTICES")
print("=" * 50)